In [1]:
from environment import WirelessEnv

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
env = WirelessEnv()

In [3]:
env.observation_space

MultiDiscrete([7 7 7 7 7 7 6])

In [4]:
env.action_space

MultiDiscrete([4 6 4 6 4 4])

In [5]:
for i in range(env.num_agents):
    act_space = env.get_agent_action_space(i)
    print("agent: ", i, "act_space", act_space)
    print(act_space.sample())

agent:  0 act_space Discrete(4)
3
agent:  1 act_space Discrete(4)
3
agent:  2 act_space Discrete(6)
4
agent:  3 act_space Discrete(6)
2
agent:  4 act_space Discrete(4)
3
agent:  5 act_space Discrete(4)
0


In [6]:
env.reset()

{0: array([1, 3, 0, 1, 6, 3, 4]),
 1: array([1, 3, 0, 1, 6, 3, 4]),
 2: array([1, 3, 0, 1, 6, 3, 4]),
 3: array([1, 3, 0, 1, 6, 3, 4]),
 4: array([1, 3, 0, 1, 6, 3, 4]),
 5: array([1, 3, 0, 1, 6, 3, 4])}

In [7]:
action_dict = {0:[1],1:[2],2:[2],3:[4],4:[2],5:[3]}
env.step(action_dict)

node, action_list, index, actions 0 [0, 1, 2, 3] 0 [1, 2, 2, 4, 2, 3]
node, action_list, index, actions 2 [0, 1, 2, 3, 4, 5] 1 [1, 2, 2, 4, 2, 3]
node, action_list, index, actions 1 [0, 1, 2, 3] 2 [1, 2, 2, 4, 2, 3]
node, action_list, index, actions 3 [0, 1, 2, 3, 4, 5] 3 [1, 2, 2, 4, 2, 3]
node, action_list, index, actions 4 [2, 3, 4, 5] 4 [1, 2, 2, 4, 2, 3]
node, action_list, index, actions 5 [2, 3, 4, 5] 5 [1, 2, 2, 4, 2, 3]


({0: array([5, 3, 0, 2, 6, 3, 4]),
  1: array([5, 3, 0, 2, 6, 3, 4]),
  2: array([5, 3, 0, 2, 6, 3, 4]),
  3: array([5, 3, 0, 2, 6, 3, 4]),
  4: array([5, 3, 0, 2, 6, 3, 4]),
  5: array([5, 3, 0, 2, 6, 3, 4])},
 {0: -210, 1: -210, 2: -210, 3: -210, 4: -210, 5: -210},
 {0: False, 1: False, 2: False, 3: False, 4: True, 5: False, '__all__': False},
 {0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}})